In [1]:
from typing import Dict, Tuple

import numpy as np
import sklearn
import sklearn.model_selection
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, Subset, DataLoader
from tqdm.notebook import tqdm
import pandas as pd

from recommend.utils import PROJ_ROOT

In [2]:
ratings = pd.read_pickle(f"{PROJ_ROOT}/data/ratings.pkl")

In [3]:
ratings.shape

(2797751, 5)

In [4]:
def filter_ratings(ratings: pd.DataFrame, min_user_ratings: int, min_movie_ratings: int) -> pd.DataFrame:
    ratings = ratings.dropna()

    user_rating_count = ratings.groupby("username").size()
    worthy_users = set(user_rating_count[user_rating_count >= min_user_ratings].index)
    ratings = ratings.loc[ratings.username.isin(worthy_users)]

    movie_rating_count = ratings.groupby("movie_id").size()
    worthy_movies = set(movie_rating_count[movie_rating_count >= min_movie_ratings].index)
    ratings = ratings.loc[ratings.movie_id.isin(worthy_movies)]
    
    return ratings

In [5]:
ratings = filter_ratings(ratings, 10, 10)

In [6]:
ratings.shape

(2722362, 5)

In [7]:
ratings_train, ratings_test = sklearn.model_selection.train_test_split(
    ratings,
    test_size=0.1,
    shuffle=True,
    random_state=0,
)

ratings_train, ratings_valid = sklearn.model_selection.train_test_split(
    ratings_train,
    test_size=0.1,
    shuffle=True,
    random_state=0,
)

In [8]:
idx2movie = pd.Series(ratings.movie_id.unique())
movie2idx = pd.Series(idx2movie.index.values, index=idx2movie.values)
movie2idx.head()

230421-houbicky                       0
10789-prvni-liga                      1
235032-yes-man                        2
234754-chlapec-v-pruhovanem-pyzamu    3
301717-nejvetsi-showman               4
dtype: int64

In [9]:
idx2user = pd.Series(ratings.username.unique())
user2idx = pd.Series(idx2user.index.values, index=idx2user.values)
user2idx.head()


kinghome     0
SimonShot    1
blackend     2
LCH          3
knoxville    4
dtype: int64

In [10]:
random_gen = np.random.default_rng(seed=42)

num_features = 100

movies_shape = len(movie2idx), num_features
movies = torch.from_numpy(
    random_gen.standard_normal(movies_shape, dtype=np.float32).clip(-2, 2)
)

users_shape = len(user2idx), num_features
users = torch.from_numpy(
    random_gen.standard_normal(users_shape, dtype=np.float32).clip(-2, 2)
)




In [11]:
device = 'cuda'
movies = movies.to(device).requires_grad_()
users = users.to(device).requires_grad_()
None

In [12]:
ratings_train.head()

,username,movie_id,stars,date,comment
328863,pepo,2596-osm-a-pul,1.0,2018-04-24,"Súhlasím, že 8 1/2 je umelecký film. Je totiž..."
1601217,Melly.pro,224137-hvezdny-prach,5.0,2008-01-18,"Tak nejsem si úplně jistej, co o tom říct, pr..."
1369474,GOREGASM,223564-kazatel,2.0,2016-01-31,Béčkové vody zakalené do digitálního apokalyp...
499592,Ernie_13,308635-zpatky-ve-hre,3.0,2014-03-07,"No je to tých 70%....hrozná réžia, hrozný sce..."
2322162,Khumbac,71194-walker-texas-ranger,4.0,2012-07-14,"Z nostalgie, z recese, z úcty ke komedii dává..."


In [13]:
class RatingDataset(Dataset): 

    def __init__(self, df_ratings: pd.DataFrame, movie2idx: pd.Series, user2idx: pd.Series):
        self.df_ratings = df_ratings
        self.movie2idx = movie2idx
        self.user2idx = user2idx

    def __len__(self) -> int:
        return len(self.df_ratings)

    def __getitem__(self, idx: int) -> Tuple[int, int, float]:
        row = self.df_ratings.iloc[idx]
        idx_movie = self.movie2idx[row.movie_id]
        idx_user = self.user2idx[row.username]
        rating = (row.stars / 5.0).astype(np.float32)
        return idx_movie, idx_user, rating

In [14]:
ds_train = RatingDataset(ratings_train, movie2idx, user2idx)
ds_valid = RatingDataset(ratings_valid, movie2idx, user2idx)
ds_test = RatingDataset(ratings_test, movie2idx, user2idx)

ds_train_mini = Subset(
    ds_train,
    np.random.default_rng(0).choice(len(ds_train), len(ds_valid), replace=False)
)

In [15]:
len(ds_train), len(ds_valid), len(ds_test), len(ds_train_mini)

(2205112, 245013, 272237, 245013)

In [16]:
ds_train[0]

(1023, 2359, 0.2)

In [17]:
batch_size = 64

def collate_move(device):
    def collate(*params):
        movie_idx, user_idx, rating = torch.utils.data.default_collate(*params)
        return movie_idx.to(device), user_idx.to(device), rating.to(device)
    return collate


loader_train = DataLoader(ds_train, batch_size, collate_fn=collate_move(device))
loader_valid = DataLoader(ds_valid, batch_size, collate_fn=collate_move(device))
loader_test = DataLoader(ds_test, batch_size, collate_fn=collate_move(device))

loader_train_mini = DataLoader(ds_train_mini, batch_size, collate_fn=collate_move(device))

In [18]:
len(loader_train), len(loader_valid), len(loader_test), len(loader_train_mini)

(34455, 3829, 4254, 3829)

In [19]:
optim = torch.optim.Adam(
    params=[movies, users],
    lr=1e-2,
)

In [23]:
class Trainer:

    def __init__(self,
        users: torch.Tensor,
        movies: torch.Tensor,
        optimizer: torch.optim.Optimizer,
    ):
        self.users = users
        self.movies = movies
        self.metrics_train = []
        self.metrics_valid = []
        self.past_movies = []
        self.past_users = []
        self.optimizer = optimizer


    def evaluate(
        self,
        loader: DataLoader,
        tqdm_desc: str,
        tqdm_leave: bool,
    ) -> Dict[str, float]:

        metrics = {
            "mse": 0.0,
            "mae_stars": 0.0
        }

        with torch.no_grad():
            for movie_idx, user_idx, rating in tqdm(loader, desc=tqdm_desc, leave=tqdm_leave, position=1):
                m = self.movies[movie_idx]
                u = self.users[user_idx]
                pred = torch.sigmoid((m * u).sum(-1))
                metrics["mse"] += F.mse_loss(pred, rating).cpu().item()
                metrics["mae_stars"] += F.l1_loss(pred * 5, rating * 5).cpu().item()
        return {name: value / len(loader) for name, value in metrics.items()}

    
    def train(
        self,
        loader_train: DataLoader,
        loader_train_mini: DataLoader,
        loader_valid: DataLoader,
        patience: int = 5,
        max_epochs: int = 25,
        evaluate_every_n_steps: int = 12500,
        early_stopping_metric: str = "mae_stars",
    ):
        step = 0
        for epoch in range(max_epochs):
            for movie_idx, user_idx, rating in tqdm(loader_train, desc=f"epoch {epoch}", position=0):
                m = self.movies[movie_idx]
                u = self.users[user_idx]
                pred = torch.sigmoid((m * u).sum(-1))
                loss = F.mse_loss(pred, rating)
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                if step % evaluate_every_n_steps == 0:
                    self.metrics_train.append(self.evaluate(loader_train_mini, "train evaluation", tqdm_leave=False))
                    self.metrics_valid.append(self.evaluate(loader_valid, "validation evaluation", tqdm_leave=False))
                    print("train metrics:     ", self.metrics_train[-1])
                    print("validation metrics:", self.metrics_valid[-1])
                    self.past_movies.append(movies.clone().detach().cpu())
                    self.past_users.append(users.clone().detach().cpu())
                    if len(self.metrics_train) >= patience:
                        self.past_movies.pop(0)
                        self.past_users.pop(0)
                        last_n = pd.DataFrame(self.metrics_train[-patience:])[early_stopping_metric]
                        if last_n.argmin() == 0:
                            # TODO revert back to best weights
                            return
                step += 1


In [24]:
trainer = Trainer(users, movies, optim)

In [25]:
trainer.train(loader_train, loader_train_mini, loader_valid)

epoch 0:   0%|          | 0/34455 [00:00<?, ?it/s]

train evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

validation evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

train metrics:      {'mse': 0.31217285436542824, 'mae_stars': 2.318140492360814}
validation metrics: {'mse': 0.31177442685668605, 'mae_stars': 2.3166307150212364}


train evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

...

validation evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

train metrics:      {'mse': 0.13752928195722142, 'mae_stars': 1.4138111741432604}
validation metrics: {'mse': 0.15384165537547242, 'mae_stars': 1.4963939515876221}


train evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

validation evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

train metrics:      {'mse': 0.13745612807554375, 'mae_stars': 1.4134965944178082}
validation metrics: {'mse': 0.15380797337679367, 'mae_stars': 1.496082435126839}
